In [26]:
import pathlib, requests, tqdm, geopandas as gpd, os

DATA = pathlib.Path(".")
GJ   = DATA / "msoa_2021.geojson"        # plain, not gzipped

URL  = ("https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Middle_layer_Super_Output_Areas_December_2021_Boundaries_EW_BGC_V3/FeatureServer/0/query?where=1%3D1&outFields=*&geometry=&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outSR=&f=json")   # ONS 20 m generalisation

if not GJ.exists():
    print("[+] Downloading MSOA boundaries …")
    with requests.get(URL, stream=True) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))
        with open(GJ, "wb") as f, tqdm.tqdm(
            total=total, unit="B", unit_scale=True, unit_divisor=1024
        ) as bar:
            for chunk in r.iter_content(chunk_size=1 << 20):
                f.write(chunk)
                bar.update(len(chunk))

size_mb = os.path.getsize(GJ) / 1_048_576
print(f"[✓] Download complete ({size_mb:.1f} MB)")

gdf = gpd.read_file(GJ)
print(f"[✓] GeoPandas read {len(gdf):,} MSOAs")


[+] Downloading MSOA boundaries …


4.74MB [00:00, 12.5MB/s]                            

[✓] Download complete (4.7 MB)
[✓] GeoPandas read 2,000 MSOAs


In [27]:
import gzip, shutil

GZ = DATA / "msoa_2021.geojson.gz"
if not GZ.exists():
    print("[+] Gzipping to save disk space …")
    with open(GJ, "rb") as src, gzip.open(GZ, "wb") as dst:
        shutil.copyfileobj(src, dst)
    print(f"[✓] Gzipped file is {GZ.stat().st_size/1_048_576:.1f} MB")


[+] Gzipping to save disk space …
[✓] Gzipped file is 12.2 MB


In [ ]:
import json
import pathlib
from functools import lru_cache
from typing import Optional

import geopandas as gpd
import numpy as np
from dash import Dash, Input, Output, html
import dash_ag_grid as dag
from dash_deckgl import DashDeckgl  # actively maintained package (pip install dash-deckgl)

# ────────────────────────────────────────────────────────────────────────────────
# 0. Quick sanity‑check: ensure dash‑deckgl is installed (≥ 0.9)
# ────────────────────────────────────────────────────────────────────────────────
try:
    import dash_deckgl as _ddg
except ImportError as e:
    raise RuntimeError(
        "dash‑deckgl missing. Run `pip uninstall -y dash-deck && pip install dash-deckgl`.") from e

# ────────────────────────────────────────────────────────────────────────────────
# 1. Load MSOA boundaries (gzipped GeoJSON) and attach demo metrics
# ────────────────────────────────────────────────────────────────────────────────
DATA = pathlib.Path('.')
GZ = DATA / 'msoa_2021.geojson.gz'

gdf = gpd.read_file(f'gzip://{GZ}')

# fabricate 3 numeric columns for demonstration
rng = np.random.default_rng(42)
gdf['employment_rate'] = rng.uniform(40, 90, len(gdf)).round(1)
gdf['years_schooling'] = rng.uniform(8, 18, len(gdf)).round(2)
gdf['median_income'] = rng.uniform(18_000, 50_000, len(gdf)).round(0)

NUM_COLS = ['employment_rate', 'years_schooling', 'median_income']

# pre‑compute centroids (project to 3857 first to avoid warning)
centroids = (
    gdf.to_crs(3857).centroid.to_crs(4326).apply(lambda p: (p.y, p.x)).to_dict()
)

PALETTE_LOW = np.array([237, 248, 177])
PALETTE_HIGH = np.array([8, 104, 172])


@lru_cache(maxsize=3)
def coloured_geojson(metric: str) -> str:
    """GeoJSON with per‑feature RGB in properties.fill (cached by metric)."""
    col = gdf[metric].astype(float)
    lo, hi = col.min(), col.max() + 1e-9
    ratio = ((col - lo) / (hi - lo)).values.reshape(-1, 1)
    rgb = (PALETTE_LOW + ratio * (PALETTE_HIGH - PALETTE_LOW)).astype(int)
    gtmp = gdf.copy()
    gtmp['fill'] = rgb.tolist()
    return gtmp[['geometry', 'MSOA21CD', 'fill'] + NUM_COLS].to_json()


def build_spec(metric: str, highlight: Optional[str] = None) -> dict:
    """Return deck.gl JSON spec given a metric and optional highlight ID."""
    view = {'latitude': 54, 'longitude': -1.5, 'zoom': 5}
    if highlight in centroids:
        lat, lon = centroids[highlight]
        view = {'latitude': lat, 'longitude': lon, 'zoom': 8}

    return {
        'initialViewState': view,
        'mapStyle': 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json',
        'layers': [
            {
                'type': 'GeoJsonLayer',
                'data': json.loads(coloured_geojson(metric)),
                'uniqueIdProperty': 'MSOA21CD',
                'pickable': True,
                'autoHighlight': True,
                'highlightedFeatureId': highlight,
                'getFillColor': '@@=properties.fill',
                'getLineColor': [150, 150, 150],
                'lineWidthMinPixels': 0.4,
            }
        ],
        'controller': True,
    }

# ────────────────────────────────────────────────────────────────────────────────
# 2. Dash components
# ────────────────────────────────────────────────────────────────────────────────
initial_metric = NUM_COLS[0]

deck_comp = DashDeckgl(
    id='deck',
    spec=build_spec(initial_metric),  # DashDeckgl expects `spec`
    width='100%',
    height='65vh',
)

grid = dag.AgGrid(
    id='tbl',
    columnDefs=[{'field': c} for c in ['MSOA21CD'] + NUM_COLS],
    rowData=gdf[['MSOA21CD'] + NUM_COLS].to_dict('records'),
    defaultColDef={'sortable': True, 'filter': True},
    dashGridOptions={'rowSelection': 'single'},
    style={'height': '33vh', 'margin': '6px 0', 'overflow': 'auto'},
)

app = Dash(__name__)
app.layout = html.Div(
    [html.Div(deck_comp, style={'flex': '0 0 65vh'}),
     html.Div(grid, style={'flex': '0 0 33vh'})],
    style={'display': 'flex', 'flexDirection': 'column', 'height': '100vh'})

# ────────────────────────────────────────────────────────────────────────────────
# 3. Callback – update DeckGL `spec` prop in milliseconds
# ────────────────────────────────────────────────────────────────────────────────

@app.callback(
    Output('deck', 'spec'),
    Input('tbl', 'columnState'),
    Input('tbl', 'selectedRows'),
)
def sync(column_state, selected_rows):
    metric = NUM_COLS[0]
    if column_state:
        sorted_cols = [c for c in column_state if c.get('sort')]
        metric = sorted_cols[0]['colId'] if sorted_cols else metric

    highlight = selected_rows[0]['MSOA21CD'] if selected_rows else None
    return build_spec(metric, highlight)

# ────────────────────────────────────────────────────────────────────────────────
# 4. Serve
# ────────────────────────────────────────────────────────────────────────────────

if __name__ == '__main__':
    app.run(debug=True, port=8050)
else:
    app.run_server(mode='external', port=8050)


Sonnet version

In [ ]:
# MSOA Interactive Map - Jupyter Notebook Implementation
# Run this in a Jupyter notebook in PyCharm

# Import necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import json
from ipyleaflet import Map, GeoJSON, LayersControl, basemaps, basemap_to_tiles
from ipywidgets import HTML, Layout, Box, VBox, HBox, Button, Output, Dropdown, interact, fixed, Tab
import ipywidgets as widgets
from IPython.display import display

# Set up sample data for demonstration
# In a real-world scenario, you would load your actual MSOA data from files

# Create sample MSOA data
msoa_data = pd.DataFrame({
    'id': [f'E0200000{i}' for i in range(1, 11)],
    'name': [f'MSOA {i:02d}' for i in range(1, 11)],
    'yearsSch': [12.5, 13.8, 13.2, 14.1, 11.9, 12.8, 13.5, 14.3, 12.2, 13.0],
    'income': [35000, 42000, 38000, 45000, 32000, 36000, 40000, 47000, 34000, 39000],
    'population': [9500, 12000, 10500, 8500, 13000, 11000, 9000, 8000, 12500, 10000]
})

# Create output widget for messages
output = Output()

# Create simple geometries for the MSOAs (simplified for demo)
# In a real implementation, you would load actual MSOA boundary files
def create_sample_geometries(msoa_data):
    features = []

    # Create a simple polygon for each MSOA
    for i, row in msoa_data.iterrows():
        # Create a square-like polygon, slightly offset for each MSOA
        center_lon = -0.1 - (i * 0.02)
        center_lat = 51.5 + (i * 0.01)

        # Simple square
        coords = [
            [center_lon - 0.01, center_lat - 0.01],
            [center_lon + 0.01, center_lat - 0.01],
            [center_lon + 0.01, center_lat + 0.01],
            [center_lon - 0.01, center_lat + 0.01],
            [center_lon - 0.01, center_lat - 0.01]
        ]

        # Create the feature
        feature = {
            "type": "Feature",
            "properties": {
                "id": row['id'],
                "name": row['name'],
                "yearsSch": row['yearsSch'],
                "income": row['income'],
                "population": row['population']
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [coords]
            }
        }
        features.append(feature)

    # Create GeoJSON object
    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }

    return geojson_data

# Create sample GeoJSON
geojson_data = create_sample_geometries(msoa_data)

# Convert to GeoDataFrame for easier manipulation
gdf = gpd.GeoDataFrame.from_features(geojson_data["features"])

# Create a color scale function for choropleth maps
def get_color_scale(values, colormap='viridis'):
    """Create a color scale based on values"""
    norm = plt.Normalize(min(values), max(values))
    cmap = plt.cm.get_cmap(colormap)

    def get_color(value):
        rgba = cmap(norm(value))
        return f'rgba({rgba[0]*255:.0f}, {rgba[1]*255:.0f}, {rgba[2]*255:.0f}, {rgba[3]:.1f})'

    return get_color

# Define style function for GeoJSON
def style_function(feature, column='yearsSch', selected_id=None):
    """Style function for GeoJSON features"""
    properties = feature['properties']
    msoa_id = properties['id']
    value = properties[column]

    # Get color based on value
    values = [f['properties'][column] for f in geojson_data['features']]
    color_func = get_color_scale(values, 'viridis')

    # Different style for selected feature
    if selected_id and msoa_id == selected_id:
        return {
            'fillColor': color_func(value),
            'color': 'black',
            'weight': 3,
            'fillOpacity': 0.7
        }
    else:
        return {
            'fillColor': color_func(value),
            'color': 'gray',
            'weight': 1,
            'fillOpacity': 0.7
        }

# Create the Jupyter notebook interface
def create_msoa_interactive_map():
    # Create base map
    m = Map(center=(51.5, -0.1), zoom=11, scroll_wheel_zoom=True)
    m.layout = Layout(width='800px', height='600px')

    # Add a tile layer
    m.add_layer(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik))

    # Variable to track the currently selected column and MSOA
    selected_column = 'yearsSch'
    selected_msoa = None

    # Create GeoJSON layer with initial styling
    geo_json = GeoJSON(
        data=geojson_data,
        style={
            'fillOpacity': 0.7,
            'weight': 1,
            'color': 'gray'
        },
        hover_style={
            'fillOpacity': 0.9,
            'weight': 2
        }
    )

    # Update layer styles based on selected column
    def update_styles():
        for feature in geo_json.data['features']:
            feature_id = feature['properties']['id']
            feature_style = style_function(feature, selected_column, selected_msoa)
            geo_json.set_feature_style(feature_id, feature_style)

    # Add GeoJSON layer to map
    m.add_layer(geo_json)

    # Create tooltip for hovering
    tooltip = HTML()
    tooltip.layout.margin = "0px 20px 20px 20px"

    # Add hover handler
    def handle_hover(feature, **kwargs):
        if feature:
            props = feature['properties']
            tooltip.value = f"""
            <div style="background-color: white; padding: 10px; border-radius: 5px; box-shadow: 0px 0px 5px gray;">
                <strong>{props['name']}</strong><br>
                {selected_column}: {props[selected_column]}
            </div>
            """
        else:
            tooltip.value = ""

    geo_json.on_hover(handle_hover)

    # Handle click events
    def handle_click(event=None, feature=None, id=None, **kwargs):
        nonlocal selected_msoa

        # If called from map click
        if feature:
            msoa_id = feature['properties']['id']
            selected_msoa = msoa_id if selected_msoa != msoa_id else None
        # If called from table selection
        elif id:
            selected_msoa = id if selected_msoa != id else None

        update_styles()
        update_table_selection()

        # If we selected an MSOA, zoom to it
        if selected_msoa:
            selected_feature = next((f for f in geo_json.data['features']
                                    if f['properties']['id'] == selected_msoa), None)
            if selected_feature:
                # Simple bounding box calculation for the selected polygon
                coords = selected_feature['geometry']['coordinates'][0]
                lats = [c[1] for c in coords]
                lons = [c[0] for c in coords]

                # Add a buffer for better visibility
                buffer = 0.02
                sw = [min(lats) - buffer, min(lons) - buffer]
                ne = [max(lats) + buffer, max(lons) + buffer]

                m.fit_bounds([sw, ne])

    geo_json.on_click(handle_click)

    # Create column selection dropdown
    column_dropdown = Dropdown(
        options=[
            ('Years of Schooling', 'yearsSch'),
            ('Income', 'income'),
            ('Population', 'population')
        ],
        value='yearsSch',
        description='Display:',
        style={'description_width': 'initial'}
    )

    # Handle column selection
    def on_column_change(change):
        nonlocal selected_column
        selected_column = change['new']
        update_styles()
        update_table()

    column_dropdown.observe(on_column_change, names='value')

    # Create reset view button
    reset_button = Button(
        description='Reset View',
        button_style='info',
        tooltip='Reset the map view'
    )

    def on_reset_button_click(b):
        nonlocal selected_msoa
        selected_msoa = None
        m.center = (51.5, -0.1)
        m.zoom = 11
        update_styles()
        update_table_selection()

    reset_button.on_click(on_reset_button_click)

    # Create controls box
    controls = HBox([column_dropdown, reset_button])

    # Create table for MSOA data
    table_output = Output()

    def update_table():
        with table_output:
            table_output.clear_output()

            # Create table header
            html = f"""
            <div style="max-height: 400px; overflow-y: auto; padding: 10px;">
            <table style="width: 100%; border-collapse: collapse;">
              <thead>
                <tr>
                  <th style="text-align: left; padding: 8px; border-bottom: 2px solid #ddd;">MSOA Name</th>
                  <th style="text-align: left; padding: 8px; border-bottom: 2px solid #ddd;">{column_dropdown.label}</th>
                </tr>
              </thead>
              <tbody>
            """

            # Add rows
            for _, row in msoa_data.iterrows():
                selected_class = "background-color: #e6f2ff;" if row['id'] == selected_msoa else ""
                html += f"""
                <tr id="{row['id']}" style="cursor: pointer; {selected_class}"
                    onclick="
                      // Send message to Python
                      var kernel = IPython.notebook.kernel;
                      var command = 'table_row_clicked(\\'{row['id']}\\')';
                      kernel.execute(command);
                    ">
                  <td style="padding: 8px; border-bottom: 1px solid #ddd;">{row['name']}</td>
                  <td style="padding: 8px; border-bottom: 1px solid #ddd;">{row[selected_column]}</td>
                </tr>
                """

            html += """
              </tbody>
            </table>
            </div>
            """
            display(HTML(html))

    # Function to update just the table selection without redrawing
    def update_table_selection():
        update_table()  # For simplicity, we'll just redraw the table

    # Register callback for table row clicks
    def table_row_clicked(id):
        handle_click(id=id)

    # Create legend
    legend_output = Output()

    def update_legend():
        with legend_output:
            legend_output.clear_output()

            # Get the values for the current column
            values = msoa_data[selected_column].tolist()
            min_val = min(values)
            max_val = max(values)

            # Create a color gradient with 5 steps
            steps = 5
            step_size = (max_val - min_val) / (steps - 1) if steps > 1 else 0

            color_func = get_color_scale(values, 'viridis')

            html = f"""
            <div style="background-color: white; padding: 10px; border-radius: 5px; box-shadow: 0px 0px 5px gray;">
                <h4 style="margin-top: 0;">{column_dropdown.label}</h4>
                <div style="display: flex; align-items: center; margin-bottom: 5px;">
                    <div style="width: 100%; height: 20px; display: flex;">
            """

            # Create gradient boxes
            for i in range(steps):
                val = min_val + (i * step_size)
                html += f"""
                <div style="flex: 1; height: 20px; background-color: {color_func(val)};"></div>
                """

            html += """
                    </div>
                </div>
                <div style="display: flex; justify-content: space-between;">
            """

            # Add min/max labels
            html += f"""
                    <span>{min_val:.1f}</span>
                    <span>{max_val:.1f}</span>
                </div>
            </div>
            """

            display(HTML(html))

    # Update legend when column changes
    def on_column_change_legend(change):
        update_legend()

    column_dropdown.observe(on_column_change_legend, names='value')

    # Initialize table and legend
    update_table()
    update_legend()

    # Create tabs for organizing the interface
    tab1 = VBox([controls, m])
    tab2 = VBox([legend_output, tooltip])
    tab3 = VBox([table_output])

    tabs = Tab(children=[tab1, tab2, tab3])
    tabs.set_title(0, 'Map')
    tabs.set_title(1, 'Legend')
    tabs.set_title(2, 'Data Table')

    # Define layout
    layout = VBox([
        HTML(value="<h2>MSOA Interactive Map</h2>"),
        HBox([
            # Left side - Map and controls
            VBox([
                controls,
                m,
                legend_output,
                tooltip
            ], layout=Layout(width='65%')),
            # Right side - Table
            VBox([
                table_output
            ], layout=Layout(width='35%'))
        ])
    ])

    return layout

# Function to run the application
def run_msoa_map():
    # Register the table click handler in the global namespace
    import IPython
    from IPython.display import display, Javascript

    # This allows the HTML onclick handlers to call back to Python
    global table_row_clicked
    table_row_clicked_out = Output()

    def table_row_clicked(id):
        with table_row_clicked_out:
            # This function will be replaced by the actual handler later
            print(f"Row clicked: {id}")

    # Create and display the map
    map_interface = create_msoa_interactive_map()
    display(map_interface)
    display(table_row_clicked_out)

    print("MSOA Interactive Map loaded successfully!")
    print("Note: This is running with sample data. In a real application, you would load actual MSOA boundaries and data.")

# Execute this to run the application
run_msoa_map()

In [69]:
# msoa_dashboard.py
import geopandas as gpd
import panel as pn
import hvplot.pandas  # adds .hvplot.geo to GeoDataFrame
from bokeh.models import HoverTool

# 1. Load data
DATA = pathlib.Path('.')
GZ = DATA / 'msoa_2021.geojson.gz'
gdf = gpd.read_file(f'gzip://{GZ}')

# fabricate 3 numeric columns for demonstration
rng = np.random.default_rng(42)
gdf['employment_rate'] = rng.uniform(40, 90, len(gdf)).round(1)
gdf['years_schooling'] = rng.uniform(8, 18, len(gdf)).round(2)
gdf['median_income'] = rng.uniform(18_000, 50_000, len(gdf)).round(0)

data_columns = ['employment_rate', 'years_schooling', 'median_income']

# 2. Create widgets
selector = pn.widgets.Select(name="Variable", options=data_columns, value=data_columns[0])
table = pn.widgets.Tabulator(
    gdf[["MSOA21CD","MSOA21NM"] + data_columns],
    selectable=1,  # single‐row select
    height=300
)

# 3. Define a function that builds the map based on state
@pn.depends(selector, table.selection)
def make_map(var, selection):
    # base choropleth
    cmap = "OrRd"
    base = gdf.hvplot.geo(
        tiles="CartoLight",
        color=var,
        cmap=cmap,
        legend="right",
        hover_cols=["MSOA21NM", var],
        frame_width=600,
        frame_height=400,
        title=f"MSOA {var.replace('_',' ').title()}"
    )

    # if a row is selected, overlay its outline
    if selection:
        code = selection[0]["MSOA21CD"]
        sel = gdf[gdf.MSOA11CD == code]
        base = base * sel.hvplot.geo(color="blue", line_width=3)

    # add hover interactivity
    return base.opts(tools=[HoverTool()])

# 4. Lay out dashboard
dashboard = pn.Column(
    "# MSOA Dashboard",
    pn.Row(selector, sizing_mode="stretch_width"),
    pn.Row(table, make_map),
    sizing_mode="stretch_width"
)

dashboard.servable()


ValueError: The depends decorator only accepts string types referencing a parameter or parameter instances, found list type instead.

In [5]:
# streamlit_app.py
import pathlib
import numpy as np
import geopandas as gpd
import streamlit as st
import folium
from streamlit_folium import st_folium
from st_aggrid import AgGrid, GridOptionsBuilder

# 1. Load data
DATA = pathlib.Path('.')
GZ = DATA / 'msoa_2021.geojson.gz'
gdf = gpd.read_file(f'gzip://{GZ}')

# fabricate 3 numeric columns for demonstration
rng = np.random.default_rng(42)
gdf['employment_rate'] = rng.uniform(40, 90, len(gdf)).round(1)
gdf['years_schooling'] = rng.uniform(8, 18, len(gdf)).round(2)
gdf['median_income'] = rng.uniform(18_000, 50_000, len(gdf)).round(0)

cols = ['employment_rate', 'years_schooling', 'median_income']

st.sidebar.title("Controls")
var = st.sidebar.selectbox("Variable", cols)

df = gdf[["MSOA21CD","MSOA21NM"]+cols]
grid = AgGrid(df, enable_enterprise_modules=False, fit_columns_on_grid_load=True)
selected = grid["selected_rows"]

m = folium.Map(location=[54, -2], zoom_start=6)
folium.Choropleth(
    geo_data=gdf,
    data=gdf,
    columns=["MSOA21CD", var],
    key_on="feature.properties.MSOA21CD",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name=var.replace("_"," ").title()
).add_to(m)

if selected:
    sf = gdf[gdf.MSOA11CD == selected[0]["MSOA21CD"]]
    folium.GeoJson(sf, style_function=lambda f: {"color":"blue","weight":3}).add_to(m)
    m.fit_bounds(sf.total_bounds.reshape(2,2).tolist())

st_folium(m, width=700)

2025-05-14 15:28:34.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 15:28:34.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

{'last_clicked': None,
 'last_object_clicked': None,
 'last_object_clicked_tooltip': None,
 'last_object_clicked_popup': None,
 'all_drawings': None,
 'last_active_drawing': None,
 'bounds': {'_southWest': {'lat': 49.8647926358421, 'lng': -6.41866722959945},
  '_northEast': {'lat': 55.8111161958565, 'lng': 1.76370560966352}},
 'zoom': 6,
 'last_circle_radius': None,
 'last_circle_polygon': None,
 'selected_layers': None}